In [1]:
import pandas as pd
import gspread
import gspread_dataframe as gd

In [2]:
def establish_connection(cred_file='./cred.json'):
    response = dict()
    response['status'] = True
    try:
        response['data'] = gspread.service_account(cred_file)
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [3]:
def get_sheet(gc: gspread.service_account, sheet, worksheet=None):
    response = dict()
    response['status'] = True
    try:
        sheet = gc.open(sheet)
        if worksheet == None:
            worksheets = sheet.worksheets()
            if len(worksheets) >= 1:
                worksheet = worksheets[0].title
            else:
                raise Exception(f"Error: No worksheet available to work with {sheet}")
        response['data'] = sheet.worksheet(worksheet)
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [4]:
def get_dataframe(csv_file):
    response = dict()
    response['status'] = True
    try:
        response['data'] = pd.read_csv(csv_file)
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [5]:
def write_to_gsheet(worksheet, df: pd.DataFrame):
    response = dict()
    response['status'] = True
    try:
        worksheet.clear()
        gd.set_with_dataframe(worksheet=worksheet,dataframe=df,include_index=False,include_column_header=True,resize=True)
        response['data'] = 'successful!'
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [6]:
def main(cred_file, csv_file, sheet, worksheet=None):
    response = dict()
    response['status'] = True
    try:
        resp = establish_connection(cred_file)
        if resp['status'] == False:
            raise Exception(resp['data'])
        
        gc = resp['data']
        
        resp = get_sheet(gc, sheet, worksheet)
        if resp['status'] == False:
            raise Exception(resp['data'])
        
        worksheet = resp['data']
        
        resp = get_dataframe(csv_file)
        if resp['status'] == False:
            raise Exception(resp['data'])
        
        df = resp['data']
        
        resp = write_to_gsheet(worksheet, df)
        if resp['status'] == False:
            raise Exception(resp['data'])
        
        response['data'] = resp['data']
        
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [7]:
cred_file = './cred.json'
csv_file = './electricity.csv'

sheet = "electricity"
worksheet = "data"

main(cred_file, csv_file, sheet, worksheet)

{'status': True, 'data': 'successful!'}